In [2]:
import numpy 
import sklearn
import pandas as pd


In [ ]:
Data_full = pd.read_csv("train.csv", index_col = "PassengerId")
features = ["Age", "CryoSleep", "VIP", 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

y = Data_full.Transported
X = Data_full.drop("Transported", axis=1)[features]


X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y)



              Age CryoSleep    VIP  RoomService  FoodCourt  ShoppingMall  \
PassengerId                                                                
7069_01      22.0     False  False          0.0      125.0           0.0   
2298_01      33.0     False  False        270.0      412.0           1.0   
4615_01      42.0     False  False         36.0        0.0        2554.0   
1451_02      39.0      True  False          0.0        0.0           0.0   
0030_01      32.0     False  False          0.0      850.0          81.0   
...           ...       ...    ...          ...        ...           ...   
8728_05      18.0      True  False          0.0        0.0           0.0   
0680_03       5.0      True    NaN          0.0        0.0           0.0   
3732_02      29.0     False  False          0.0     8444.0           0.0   
5205_01      22.0     False  False        261.0        0.0           0.0   
5040_01      26.0     False  False          0.0       21.0       10424.0   

           

Data cleaning (minimal in this version)

In [7]:
# clean NA

for col in X_train.columns:
    print(col, X_train[col].isna().sum())

Age 133
CryoSleep 163
VIP 151
RoomService 141
FoodCourt 135
ShoppingMall 153
Spa 140
VRDeck 143
